<a href="https://colab.research.google.com/github/jsg31/TS-AD-with-2D-CAEs/blob/master/TSAD_Images_SP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Anomaly Detection with Image Encodings

This colab is built from the code developed by Gabriel Rodriguez Garcia in the frame of the collaboration research work made between Airbus AI Research and the Chair of Intelligent Maintenance Systems of ETH Zurich. 

ETH Zurich:
  - Pr. Olga Fink
  - Dr. Gabriel Michau
  - **Gabriel Rodriguez Garcia**

Airbus AI Research
  - Mélanie Ducoffe
  - Jayant Sen Gupta
  
  
The idea is to transform (1D) time series data into images and use anomaly detection techniques on images to spot anomalies.  

*note: code is not yet put into a python, so it is a bit dirty*

In [1]:
!git clone https://github.com/gabriel-rodriguez-garcia/TS-AD-with-2D-CAEs.git
!pip3 install pyts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from skimage.measure import block_reduce
import os


Cloning into 'TS-AD-with-2D-CAEs'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 275 (delta 9), reused 0 (delta 0), pack-reused 253
Receiving objects: 100% (275/275), 791.51 KiB | 2.08 MiB/s, done.
Resolving deltas: 100% (112/112), done.
     |████████████████████████████████| 2.5MB 6.7MB/s 


In [2]:
!pip3 install wget
import wget
url = 'https://advised-public-data.s3-eu-west-1.amazonaws.com/dftrain.h5'
filename = wget.download(url)
filename
X = np.array(pd.read_hdf(filename))
print('Data set shape:',np.shape(X))

url2 = 'https://advised-public-data.s3-eu-west-1.amazonaws.com/dfvalid.h5'
filename2 = wget.download(url2)
filename2
Xtest = np.array(pd.read_hdf(filename2))
print('Data set shape:',np.shape(Xtest))



  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=018c0030f8ebd77bc3bd256df77efc2b358f54432471f5100ad394b904b80952
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Data set shape: (1677, 61440)
Data set shape: (594, 61440)


In [0]:
from scipy import signal
from skimage.measure import block_reduce
import os
from PIL import Image


""" The function returns a 3D matrix.
    The new 3D matrix contains several 2D matrices, which correspond to the time series encodings/images.
    The order of the objects does not change, which means for example that the 23rd slice of the 
    input dataset corresponds to the 23rd encoding in the 3D Matrix.

    The images in this case are spectrograms. Note for this encoding, slicing is done after encoding.
    Output Image shape is fixed at 64x64"""

dataset = X
sampling_freq_hz = 1024
window = 'hanning'
nperseg=126 
nfft=None 
noverlap=118 
padded=False
n,l = np.shape(dataset)
print('Encoding started...')

for p in range(0,n):

  if p==0:
    _,_,sample = signal.stft(dataset[p,:], fs=sampling_freq_hz,window=window,nperseg=nperseg, nfft=nfft, noverlap=noverlap, padded=padded)
    X_sp=np.zeros((n,64,np.shape(sample)[1]))

  _,_,spec = signal.stft(dataset[p,:], fs=sampling_freq_hz,window=window,nperseg=nperseg, nfft=nfft, noverlap=noverlap, padded=padded)
  spec = np.abs(spec)**2
  spec=np.where(spec==0.0, 0.0001, spec)
  spec=10*np.log10(spec) # dB Scale

  X_sp[p]=spec


print('Encoding successful!')
print('#####################################')
n_slices = 120
n = np.shape(X_sp)[0]
X_sp_red=np.zeros((n*n_slices,64,64))
for p in range(0,n):
  j=0
  for i in range (p*n_slices,(p+1)*n_slices):
    X_sp_red[i] = encoding_dataset[p,:,j*64:(j+1)*64]
    j+=1

print('output 3D Matrix shape:',np.shape(X_sp_red))
print('#####################################')

if not os.path.exists('SP'):
  os.makedirs('SP')

directory = 'SP/SP_Images'
dataset_name = 'X_sp.npy'


if not os.path.exists(directory):
    os.makedirs(directory)

print('Saving data...')
np.save(os.path.join(directory,dataset_name),dataset)
print(dataset_name,' saved at: ',os.path.join(directory,dataset_name))



Encoding started...
Encoding successful!
#####################################


In [0]:
"""
same for Xtest
""""

dataset = Xtest
sampling_freq_hz = 1024
window = 'hanning'
nperseg=126 
nfft=None 
noverlap=118 
padded=False
n,l = np.shape(dataset)
print('Encoding started...')

for p in range(0,n):

  if p==0:
    _,_,sample = signal.stft(dataset[p,:], fs=sampling_freq_hz,window=window,nperseg=nperseg, nfft=nfft, noverlap=noverlap, padded=padded)
    X_sp=np.zeros((n,64,np.shape(sample)[1]))

  _,_,spec = signal.stft(dataset[p,:], fs=sampling_freq_hz,window=window,nperseg=nperseg, nfft=nfft, noverlap=noverlap, padded=padded)
  spec = np.abs(spec)**2
  spec=np.where(spec==0.0, 0.0001, spec)
  spec=10*np.log10(spec) # dB Scale

  X_sp[p]=spec


print('Encoding successful!')
print('#####################################')
n_slices = 120
n = np.shape(X_sp)[0]
X_sp_red=np.zeros((n*n_slices,64,64))
for p in range(0,n):
  j=0
  for i in range (p*n_slices,(p+1)*n_slices):
    X_sp_red[i] = encoding_dataset[p,:,j*64:(j+1)*64]
    j+=1

print('output 3D Matrix shape:',np.shape(X_sp_red))
print('#####################################')

if not os.path.exists('SP'):
  os.makedirs('SP')

directory = 'SP/SP_Images'
dataset_name = 'X_sp.npy'


if not os.path.exists(directory):
    os.makedirs(directory)

print('Saving data...')
np.save(os.path.join(directory,dataset_name),dataset)
print(dataset_name,' saved at: ',os.path.join(directory,dataset_name))

